### This notebook is not updated to reflect the last additions to [this other notebook](IEEE39_with_stochastic_load.ipynb).

In [ ]:
import os
import sys
import numpy as np
from numpy.random import RandomState, SeedSequence, MT19937
import matplotlib.pyplot as plt

powerfactory_path = r'C:\Program Files\DIgSILENT\PowerFactory 2020 SP4\Python\3.8'
if powerfactory_path not in sys.path:
    sys.path.append(powerfactory_path)
import powerfactory as pf

from pfcommon import *

In [ ]:
app = pf.GetApplication()
if app is None:
    raise Exception('Cannot get PowerFactory application')
else:
    print('Successfully obtained PowerFactory application.')

In [ ]:
project_name = '\\Terna_Inerzia\\39 Bus New England System'
err = app.ActivateProject(project_name)
if err:
    raise Exception(f'Cannot activate project {project_name}')
print(f'Successfully activated project {project_name}.')

In [ ]:
project = app.GetActiveProject()
if project is None:
    raise Exception('Cannot get active project')
print('Successfully obtained active project.')

In [ ]:
project_folders = {}
for folder_name in ('study',):
    project_folders[folder_name] = app.GetProjectFolder(folder_name)
    if project_folders[folder_name] is None:
        raise Exception(f'No folder "{folder_name}" present')
    print(f'Successfully obtained folder "{folder_name}".')

In [ ]:
generators = app.GetCalcRelevantObjects('*.ElmSym')
lines = app.GetCalcRelevantObjects('*.ElmLne')
buses = app.GetCalcRelevantObjects('*.ElmTerm')
loads = app.GetCalcRelevantObjects('*.ElmLod')
n_generators, n_lines, n_buses, n_loads = len(generators), len(lines), len(buses), len(loads)
print(f'There are {n_generators} generators.')
print(f'There are {n_lines} lines.')
print(f'There are {n_buses} buses.')
print(f'There are {n_loads} loads.')

In [ ]:
for line in lines:
    line_type = line.GetAttribute('typ_id')
    vrating = line_type.GetAttribute('uline')
    print(f'{line.loc_name}: Vrating = {vrating:6.1f} kV.')

In [ ]:
P = {}
S = {}
H = {}
for generator in generators:
    i = int(generator.loc_name.split(' ')[1])
    generator_type = generator.GetAttribute('typ_id')
    P[i] = generator.GetAttribute('pgini')
    S[i] = generator_type.GetAttribute('sgn')
    H[i] = generator_type.GetAttribute('h')
    print(f'{generator.loc_name}: P = {P[i]:4.0f} MW, S = {S[i]:5.0f} MVA, inertia = {H[i]:5.2f} s.')

In [ ]:
areas_map = {
    1: [2, 3, 10],
    2: [4, 5, 6, 7],
    3: [8, 9],
    4: [1]
}
Harea = {}   # inertia
Earea = {}   # energy
Marea = {}   # momentum
for area_id,generator_ids in areas_map.items():
    num, den = 0,0
    for generator_id in generator_ids:
        num += S[generator_id] * H[generator_id]
        den += S[generator_id]
    Harea[area_id] = num / den 
    Earea[area_id] = num * 1e-3
    Marea[area_id] = 2 * num * 1e-3 / 60
print('Area inertias:  [{}] s.'.format(', '.join(list(map(lambda s: f'{s:5.2f}', Harea.values())))))
print('Area energies:  [{}] GW s.'.format(', '.join(list(map(lambda s: f'{s:5.2f}', Earea.values())))))
print('Area momentums: [{}] GW s^2.'.format(', '.join(list(map(lambda s: f'{s:5.2f}', Marea.values())))))

## Load flow analysis

In [ ]:
study_case_name = '1. Power Flow'
study_case = project_folders['study'].GetContents(study_case_name)[0]
err = study_case.Activate() # don't know why this returns 1
# if err:
#     raise Exception(f'Cannot activate study case {study_case_name}')
print(f'Successfully activated study case {study_case_name}.')

In [ ]:
load_flow = app.GetFromStudyCase('ComLdf')
err = load_flow.Execute()
if err:
    raise Exception(f'Cannot run load flow')
print(f'Successfully run load flow.')

In [ ]:
Ptot = 0
Qtot = 0
for gen in generators:
    P = gen.GetAttribute('m:Psum:bus1')
    Q = gen.GetAttribute('m:Qsum:bus1')
    I = gen.GetAttribute('m:I:bus1')
    Ptot += P
    Qtot += Q
    print(f'{gen.loc_name}: P = {P:7.2f} MW, Q = {Q:6.2f} MVAR, I = {I:6.3f} kA.')
print(f'Total P = {Ptot*1e-3:5.2f} GW, total Q = {Qtot*1e-3:5.2f} GVAR')

In [ ]:
Ptot = 0
Qtot = 0
for load in loads:
    P = load.GetAttribute('m:Psum:bus1')
    Q = load.GetAttribute('m:Qsum:bus1')
    I = load.GetAttribute('m:I:bus1')
    V = load.GetAttribute('m:U1:bus1')    # line-to-ground voltage
    Vl = load.GetAttribute('m:U1l:bus1')  # line-to-line voltage
    Ptot += P
    Qtot += Q
    print(f'{load.loc_name}: P = {P:7.2f} MW, Q = {Q:6.2f} MVAR, I = {I:6.3f} kA, V = {V:8.3f} kV.')
print(f'Total P = {Ptot*1e-3:5.2f} GW, total Q = {Qtot*1e-3:5.2f} GVAR')

In [ ]:
power_types = ['gen','load','flow','out']
print('Power types: (' + ' '.join(power_types) + ')')
for bus in buses:
    P = {power_type: bus.GetAttribute(f'm:P{power_type}') for power_type in power_types}
    Q = {power_type: bus.GetAttribute(f'm:Q{power_type}') for power_type in power_types}
    V = bus.GetAttribute('m:U')
    Pline = ' '.join([f'{P[power_type]:6.1f}' for power_type in power_types])
    Qline = ' '.join([f'{Q[power_type]:6.1f}' for power_type in power_types])
    print(f'{bus.loc_name}: V = {V:8.3f} kV, (' + Pline + ') MW, (' + Qline + ') MVar')

## Transient stability analysis

In [ ]:
study_case_name = '5. Transient Stability'
if '.IntCase' not in study_case_name and False:
    study_case_name += '.IntCase'
study_case = project_folders['study'].GetContents(study_case_name)[0]
err = study_case.Activate() # don't know why this returns 1
# if err:
#     raise Exception(f'Cannot activate study case {study_case_name}')
print(f'Successfully activated study case {study_case_name}.')

Objects that will be used in the following:

In [ ]:
generators = app.GetCalcRelevantObjects('*.ElmSym')
loads = app.GetCalcRelevantObjects('*.ElmLod')
buses = app.GetCalcRelevantObjects('*.ElmTerm')

In [ ]:
monitored_variables = {
    '*.ElmSym': ['s:xspeed'],
    '*.ElmLod': ['m:Psum:bus1', 'm:Qsum:bus1'],
    '*.ElmTerm': ['m:u', 'm:ur', 'm:ui', 'm:u1', 'm:u1r', 'm:u1i', 'm:Pflow', 'm:Qflow', 'm:Pout', 'm:Qout']
}
# the results of the transient simulation will be stored in this variable
res = app.GetFromStudyCase('*.ElmRes')
for elements,var_names in monitored_variables.items():
    for element in app.GetCalcRelevantObjects(elements):
        for var_name in var_names:
            res.AddVariable(element, var_name)

In [ ]:
dt = 10 # [ms]
inc = app.GetFromStudyCase('ComInc')
inc.iopt_sim = 'rms'
inc.tstart = 0
inc.dtgrd = dt
err = inc.Execute()
if err:
    raise Exception('Cannot compute initial condition')
print('Successfully computed initial condition.')

In [ ]:
sim = app.GetFromStudyCase('ComSim')
sim.tstop = 0.5
# sim.dtstop = dt
err = sim.Execute()
if err:
    raise Exception('Cannot run transient simulation')
print('Successfully run transient simulation.')
res.Load()

Get the data:

In [ ]:
# dtsim = get_simulation_dt(res) # [s]
# dec = int(dt // dtsim)
dec = 1

sys.stdout.write('.')
sys.stdout.flush()
time = get_simulation_time(res, decimation=dec)

sys.stdout.write('.')
sys.stdout.flush()
speed = get_simulation_variables(res, 's:xspeed', elements=generators, decimation=dec)

sys.stdout.write('.')
sys.stdout.flush()
P = get_simulation_variables(res, 'm:Psum:bus1', elements=loads, decimation=dec)

sys.stdout.write('.')
sys.stdout.flush()
Q = get_simulation_variables(res, 'm:Qsum:bus1', elements=loads, decimation=dec)

sys.stdout.write('.')
sys.stdout.flush()
V = get_simulation_variables(res, 'm:u', elements=buses, decimation=dec)

sys.stdout.write('.')
sys.stdout.flush()
Vr = get_simulation_variables(res, 'm:ur', elements=buses, decimation=dec)

sys.stdout.write('.')
sys.stdout.flush()
Vi = get_simulation_variables(res, 'm:ui', elements=buses, decimation=dec)

In [ ]:
cmap = plt.get_cmap('Set2', n_generators)
fig,ax = plt.subplots(1, 1, figsize=(8,4))
if len(speed.shape) > 1:
    for i,generator in enumerate(generators):
        ax.plot(time, speed[:,i], color=cmap(i), lw=1, label=generator.loc_name)
else:
    ax.plot(time, speed, color='k', lw=1, label=generator.loc_name)
ax.legend(loc='upper right')
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Speed [p.u.]')
ax.grid(which='major', axis='both', lw=0.5, ls=':')
fig.tight_layout()

In [ ]:
cmap = plt.get_cmap('Set2', len(buses))
fig,ax = plt.subplots(2, 1, figsize=(8,5), sharex=True)
for i,load in enumerate(loads):
    ax[0].plot(time, P[:,i], color=cmap(i), lw=1, label=load.loc_name)
    ax[1].plot(time, Q[:,i], color=cmap(i), lw=1, label=load.loc_name)
# ax[0].legend(loc='upper left')
for a in ax:
    a.grid(which='major', axis='both', lw=0.5, ls=':')
    for side in 'right','top':
        a.spines[side].set_visible(False)
ax[1].set_xlabel('Time [s]')
ax[0].set_ylabel('P [MW]')
ax[1].set_ylabel('Q [MVAR]')
fig.tight_layout()

In [ ]:
cmap = plt.get_cmap('Set2', len(buses))
fig,ax = plt.subplots(3, 1, figsize=(8,7), sharex=True)
for i,load in enumerate(buses):
    ax[0].plot(time, V[:,i], color=cmap(i), lw=1, label=load.loc_name)
    ax[1].plot(time, Vr[:,i], color=cmap(i), lw=1, label=load.loc_name)
    ax[2].plot(time, Vi[:,i], color=cmap(i), lw=1, label=load.loc_name)
# ax[0].legend(loc='upper left')
for a in ax:
    a.grid(which='major', axis='both', lw=0.5, ls=':')
    for side in 'right','top':
        a.spines[side].set_visible(False)
ax[-1].set_xlabel('Time [s]')
ax[0].set_ylabel('V [p.u.]')
ax[1].set_ylabel('Vr [p.u.]')
ax[2].set_ylabel('Vi [p.u.]')
fig.tight_layout()